# Ask Acme Demo

This notebook demonstrates the Ask Acme agent - a LangGraph orchestrator that answers questions about Acme Corp by routing to the right data sources.

## Setup

Make sure you've:
1. Started the databases: `docker compose up -d` (from `setup_environment/`)
2. Seeded the data: `uv run python seed_all.py` (from `setup_seed_data/`)
3. Configured your `.env` file with `OPENAI_API_KEY`

In [1]:
# Import the agent
from orchestrator import ask_acme, ask_acme_with_trace

## Single-Tool Queries

Let's test each tool individually.

### Vector Search (Documents)

In [2]:
# Query about policies
response = ask_acme("What is our remote work policy?")
print(response)

Acme’s Remote Work Policy (remote-first)

1) Work location options
- Acme is remote-first. Employees may work fully remote, fully from the office, or hybrid, depending on preferences and any role requirements.
Source: remote_work_policy.md (“Overview”)

2) Core hours (availability expectation)
- Everyone should be available during core hours: 10:00 AM–3:00 PM Pacific Time, Monday–Friday.
- Outside core hours, you can structure your day flexibly.
Source: remote_work_policy.md (“Core Hours”)

3) Home office support
- Equipment provided: laptop (refresh every 3 years), one 27” monitor, keyboard/mouse, and headset.
- Home office stipend: $500 annually (paid in January; does not roll over). Can be used for ergonomic furniture, internet upgrade, supplies, lighting, etc.
Source: remote_work_policy.md (“Home Office Setup”)

4) Communication expectations
- Slack: respond within 4 hours during core hours.
- Email: respond within 24 hours.
- Urgent issues: use phone or Slack with @mention.
- Vide

### SQL Agent (Metrics)

In [3]:
# Query about customers
response = ask_acme("How many active customers do we have?")
print(response)

As of 2026-01-26 17:30:46 UTC, we have 22 active customers.

Definition used: customers with customers.status = 'active'.

Source: Analytics DB (sql_query against customers table).


### Jira Search (Tickets)

In [4]:
# Query about bugs
response = ask_acme("What P0 or P1 bugs are open?")
print(response)

Open P0/P1 bugs (per Jira):

1) MOB-892 (P1) — Open
   Summary: iOS app crashes when opening project with 500+ tasks
   Assignee: Unassigned
   Labels: crash, performance, ios

No P0 bugs are currently open in Jira based on this search; the only P0 returned (PLAT-1247) is Resolved.
Sources: Jira MOB-892; Jira PLAT-1247


### Slack Search (Discussions)

In [5]:
# Query about decisions
response = ask_acme("What did engineering decide about the notification service?")
print(response)

Engineering decided to migrate the notification service to Kafka (instead of keeping Postgres and adding partitioning).

Rationale captured in the thread:
- Postgres is already hitting limits at ~10k notifications/min, and mobile push is expected to drive ~10x volume.
- Kafka would provide higher throughput and replay capability for debugging.

Ownership/timing:
- Decision/consensus recorded on 2025-01-09; James Liu will lead.
- Plan was to kick off the work next sprint.

Sources:
- Slack: #engineering (messages from Sarah Chen, James Liu, Priya Patel, Alex Rivera on 2025-01-08 to 2025-01-09)
- Jira: I didn’t find a specific “notification service rewrite” ticket via search (only related notifications bug MOB-890 showed up).


## Multi-Tool Queries

These queries require multiple tools to answer completely.

In [6]:
# Policy + Metrics query (uses both vector_search and sql_query)
response = ask_acme("What's our refund policy and how many refunds did we process in Q4 2024?")
print(response)

Refund policy (Customer Refund Policy; effective Jan 1, 2024; last updated Oct 1, 2024):

- Full refunds if:
  - Customer cancels within 30 days of initial subscription, or
  - They experienced an outage exceeding our SLA commitments (99.9% uptime), or
  - They were incorrectly billed due to our error.
- Pro-rata refunds if:
  - Customer cancels an annual subscription mid-term after the 30-day window, or
  - They downgrade from a higher tier mid-billing cycle.
- No refunds for:
  - Monthly subscriptions (can cancel anytime, but no refund for partial months)
  - Usage-based charges already consumed
  - Add-ons (storage, API calls) already used
- Process/timing:
  - CS responds within 2 business days; approved refunds are processed within 5–7 business days.
- Note: Enterprise customers may have different terms per their MSA/contract.

Source: refund_policy.md (“Customer Refund Policy”, last updated Oct 1, 2024)

Refunds processed in Q4 2024 (Oct 1–Dec 31, 2024):
- Refund transactions pro

In [ ]:
# Let's see the trace for this query
trace = ask_acme_with_trace("What's our refund policy and how many refunds did we process in Q4 2024?")

print("Tools called:")
for tc in trace["tool_calls"]:
    print(f"  - {tc['tool']}: {tc['args']}")

print(f"\nTotal messages: {trace['message_count']}")

## Interactive Mode

Ask your own questions!

In [ ]:
# Type your question here
question = "What's our pricing strategy?"
response = ask_acme(question)
print(response)

## Next Steps

Now that the agent is working, let's evaluate it properly!

Continue to **Stage 1: Golden Sets** to create your first evaluation suite.

```bash
cd ../stage_1_golden_sets
cat README.md
```